# Working with Data - CLI Tools

### jq

> `jq` is like `sed` for JSON data - you can use it to slice and filter and map and transform structured data with the same ease that `sed`, `awk`, `grep` and friends let you play with text.

https://jqlang.github.io/jq/

##### Request the JSON users content from the previous example and pass it to `jq`

In [ ]:
curl -s 'https://jsonplaceholder.typicode.com/users' | jq .

##### Read the `basic_example.json` file

In [ ]:
jq . basic_example.json

##### Another way to read in the `basic_example.json` file

In [ ]:
cat basic_example.json | jq

##### Return all items of the list 

In [ ]:
jq '.[]' basic_example.json

##### Return the first item or index 0 from the list

In [ ]:
jq '.[0]' basic_example.json

##### Get the value of a key called `name` in the first list item

In [ ]:
jq '.[0].name' basic_example.json

##### Get all names from the list

In [ ]:
jq '.[].name' basic_example.json

##### Create an object with the values of the `id` and `name` keys

In [ ]:
jq '.[] | {id, name}' basic_example.json

##### Join the IDs and Names in the format `id - name`

In [ ]:
jq  '.[] | {id, name} | join (" - ")' basic_example.json

##### Remove the quotes

In [ ]:
jq --help | grep raw
echo
jq -r '.[] | {id, name} | join (" - ")' basic_example.json

##### Only return items which match a criteria

In [ ]:
jq '.[] | select((.name == "Ervin Howell") or ( .name | contains("Leanne")))' basic_example.json

##### Only return items which match a criteria and then only print the value of the `address` key in that item

In [ ]:
jq '.[] | select(.name == "Ervin Howell") | {address}' basic_example.json

##### Only return items which match a criteria and then only print all keys foudn in `address` 

In [ ]:
jq '.[] | select(.name == "Ervin Howell") | {address} | .address | keys '  basic_example.json

##### Only return items which match a criteria and then print the value of a specific key in that item

In [ ]:
jq '.[] | select(.name == "Ervin Howell") | {address} | .address.city ' basic_example.json

##### Show the JSON for a new user

In [ ]:
cat new_user.json

##### Append it as an item to the list in `basic_example.json`

In [ ]:
jq '. += [input]' basic_example.json new_user.json

##### Notice that `basic_example.json` does not contain the new user?

In [ ]:
cat basic_example.json

##### Write the new appended list to a temporary file and print the contents

In [ ]:
jq '. += [input]' basic_example.json new_user.json > temp_list_of_users.json
cat temp_list_of_users.json

##### Use this temporary file in a shell script

In [ ]:
echo -e "Using jq in a shell script \n"

list_of_names=$(jq -r '.[].name | @sh' temp_list_of_users.json)

for i in "${list_of_names[@]}"; do
    echo "$i"
done


"The `@foo` syntax is used to format and escape strings, which is useful for building URLs, documents in a language like HTML or XML, and so forth."

"`@sh`:
The input is escaped suitable for use in a command-line for a POSIX shell. If the input is an array, the output will be a series of space-separated strings."

https://devdocs.io/jq/

"`@`	Used in filter expressions to refer to the current node being processed."

https://support.smartbear.com/alertsite/docs/monitors/api/endpoint/jsonpath.html


##### Remove the temporary file

In [ ]:
rm temp_list_of_users.json

##### Working with Kubernetes and `kubectl`

The `kubernetes-nodes.json` and `kubernetes-pods.json` files were created using the `kubectl get nodes -json` and `kubectl get pods --all-namespaces -json` commands

You can also pipe this output straight into `jq`

e.g. ` kubectl get pods --all-namespaces -o json | jq '.'`

In [ ]:
cat kubernetes-nodes.json | jq '.'

##### Find the Kubernetes node name

In [ ]:
cat kubernetes-nodes.json | jq '.items[] | .metadata.name'

##### Create an object containing the name and status of each node

In [ ]:
cat kubernetes-nodes.json | jq '.items[] |{name: .metadata.name, conditions: .status.conditions}'

##### Filter to just the name, status, and message

In [ ]:
cat kubernetes-nodes.json | jq '.items[] |{name: .metadata.name, status: .status.conditions[].status, message: .status.conditions[].message} '

##### It can be consolidated further. Note the array `[]` around the conditions

In [ ]:
cat kubernetes-nodes.json | jq '.items[] |{name: .metadata.name, conditions: [.status.conditions[] | {status, message}]} '

##### Have a look at the Kubernetes pods

In [ ]:
cat kubernetes-pods.json | jq '.items[]'

##### Find the failing pods

In [ ]:
cat kubernetes-pods.json | jq '.items[] | select(.status.phase != "Running") | {name: .metadata.name, namespace: .metadata.namespace, status: .status.phase, reason: .status.containerStatuses[].state[].reason, message: .status.containerStatuses[].state[].message} '

### A few more CLI examples

##### Using `tr` to transform - lowercase to UPPERCASE

In [ ]:
cat kubernetes-nodes.json | jq -r '.items[] | .metadata.name' 
echo
cat kubernetes-nodes.json | jq -r '.items[] | .metadata.name' | tr [:lower:] [:upper:]

##### Using `tr` to transform - replace hyphen with underscore

In [ ]:
cat kubernetes-nodes.json | jq -r '.items[] | .metadata.name' | tr '-' '_'

##### Using `tr` to transform - delete all the numbers

In [ ]:
cat kubernetes-nodes.json | jq -r '.items[] | .metadata.name' | tr -d '[:digit:]'

##### Using `cut` to select data

> The ` kubectl-pod-output` file was created using the command, `kubectl get pods -A > kubectl-pod-output`

In [ ]:
cat kubectl-pod-output

In [ ]:
cat kubectl-pod-output | cut -c 1

In [ ]:
cat kubectl-pod-output | tr -s ' ' | cut -d ' ' -f 2,3

##### It's easier with `awk`

This is only a fraction of a fraction of what is possible with `awk`. If you want to see what's possible have a look at tutorials such as this one:

[Getting started with AWK](https://www.gnu.org/software/gawk/manual/gawk.html#Getting-Started)


In [ ]:
cat kubectl-pod-output | awk '{print $2,$3}'

[Gawk: Effective AWK Programming](https://www.gnu.org/software/gawk/manual/)

- `printf "%-30s %-100s\n",$1,$2`: pretty print whats between the quotes, `" "`, and insert the values in variables `$1` and `$2`

- `printf`: you can specify the width to use for each item, as well as various formatting choices for numbers (such as what output base to use, whether to print an exponent, whether to print a sign, and how many digits to print after the decimal point)
- `%-30s`: left align the string value in variable `$1` and make the column 30 characters wide
- `%-100s`: left align the string value in variable `$2` and make the column 30 characters wide
- `\n`: newline character
- `$1,$2`: variables holding data


> Try removing the hyphen, `-`, or changing the 30 and 100 to different values to see what happens. 


In [ ]:
cat kubectl-pod-output | awk '{printf "%-30s %-100s\n",$1,$2}'